# Running ASFMAUS

In [ ]:
using Distributed

In [ ]:
addprocs(5)

In [ ]:
nprocs()

In [1]:
using StochasticDiffEq
using DiffEqCallbacks
using DifferentialEquations
 using LinearAlgebra
using Statistics
using SparseArrays

using Plots
using DelimitedFiles

In [47]:
 include("/home/callum/ASF/Modules/ASF_input.jl");
include("/home/callum/ASF/Modules/ASF_analysis.jl");

 include("/home/callum/ASF/Modules/ASF_model.jl");

In [4]:
input_path = "/home/callum/ASF/Inputs/";

In [39]:
input_1 = ASF_Inputs.Model_Data(input_path); #creating parameters to run model

Watts Strogatz Small Worlds Network
Random: 1
Warning 1 disconnected feral groups!


In [43]:
 nt = 500#input_1.Parameters.Populations.cum_sum[end] #total number of groups and farms
 nc = 5 #number of classes
 eqs = 11 #number of processes

 dc = sparse(zeros(nt*nc,nt*eqs))

#const _dc = zeros(nt*5,nt*10)

dc[0*nc*nt+1:nc*nt*eqs+nc:end] .= 1
dc[1*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[2*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[10*nc*nt+1:nc*nt*eqs+nc:end] .= 1

dc[2*nc*nt+2:nc*nt*eqs+nc:end] .= 1
dc[3*nc*nt+2:nc*nt*eqs+nc:end] .= -1
dc[4*nc*nt+2:nc*nt*eqs+nc:end] .= -1

dc[4*nc*nt+3:nc*nt*eqs+nc:end] .= 1
dc[5*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[6*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[7*nc*nt+3:nc*nt*eqs+nc:end] .= -1

dc[7*nc*nt+4:nc*nt*eqs+nc:end] .= 1
dc[8*nc*nt+4:nc*nt*eqs+nc:end] .= -1
dc[10*nc*nt+4:nc*nt*eqs+nc:end] .= -1


dc[5*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[6*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[9*nc*nt+5:nc*nt*eqs+nc:end] .= -1;

In [36]:
input_1.Populations_data[1].N_feral

2-element Vector{UInt16}:
 0x0003
 0x0000

In [44]:
 function regular_c(du,u,p,t,counts,mark)  
    mul!(du,dc,counts)
    nothing
end

function prob_func(prob,i,repeat) #function to change parameters while keeping network the same
    ASF_Model.reparam!(input_1)
    remake(prob,p=input_1.Parameters)
end 

prob_func (generic function with 1 method)

## Running the Model

### Frequency run

In [45]:
input_1 = ASF_Inputs.Model_Data(input_path); #creating parameters to run model

Watts Strogatz Small Worlds Network
Random: 1
Warning 1 disconnected feral groups!


In [59]:

N_years = 5

 rj = RegularJump(ASF_Model.density_rate_single,regular_c,eqs*nt)

 jumps = JumpSet(rj)

 prob = DiscreteProblem(init_pop,(0.0,365*N_years),input_1.Parameters)

 jump_prob = JumpProblem(prob,Direct(),rj) #rng=rng)

sol = solve(jump_prob, SimpleTauLeaping(),dt=1)
 #ensemble_prob_m = EnsembleProblem(jump_prob)#,prob_func=prob_func)


retcode: Default
Interpolation: Piecewise constant interpolation
t: 0.0:1.0:1825.0
u: 1826-element Vector{Vector{Int64}}:
 [4090, 5, 5, 0, 0]
 [4089, 5, 6, 0, 1]
 [4085, 8, 6, 0, 2]
 [4069, 19, 5, 0, 3]
 [4058, 19, 8, 0, 4]
 [4045, 26, 7, 1, 5]
 [4036, 27, 10, 2, 5]
 [4031, 31, 16, 2, 7]
 [4018, 27, 25, 2, 8]
 [4003, 35, 30, 2, 10]
 [3975, 50, 27, 3, 16]
 [3949, 74, 26, 4, 21]
 [3921, 80, 39, 5, 24]
 ⋮
 [0, 2, 0, 9, 3]
 [0, 2, 0, 9, 3]
 [0, 2, 0, 9, 3]
 [0, 2, 0, 9, 3]
 [0, 2, 0, 9, 3]
 [0, 2, 0, 8, 3]
 [0, 2, 0, 8, 3]
 [0, 2, 0, 8, 3]
 [0, 2, 0, 8, 2]
 [0, 0, 2, 8, 2]
 [0, 0, 1, 9, 2]
 [0, 0, 1, 9, 2]

In [ ]:
Threads.nthreads()

In [ ]:
solve(ensemble_prob_m, SimpleTauLeaping(),trajectories= 10,EnsembleSplitThreads(), dt=1);

In [ ]:
Threads.nthreads()

In [ ]:
data = reduce(vcat,transpose.(sol.u))
    
if any(x->x <0, data)
    println("Need to Reduce Timestep")
    data[data .< 0 ] .= 0
end

s_d = data[:,1:5:end]
e_d = data[:,2:5:end]
i_d = data[:,3:5:end]
r_d = data[:,4:5:end]
c_d = data[:,5:5:end]

disease = e_d + i_d + c_d #classes with disease
disease_free = s_d + r_d #classes without disease

disease_sum = sum(disease,dims=2);
pop_sum = sum(disease_free,dims=2);

In [ ]:
plot(pop_sum)
plot!(disease_sum)

In [ ]:
input_1 = ASF_Inputs.Model_Data(input_path); #creating parameters to run model
prob = ODEProblem(SEIRC_ODE!,input_1.U0,(0.0,365.0*N_years),input_1.Parameters);
sol=solve(prob, QNDF(),saveat=1);


In [68]:
function sensitivity3(n_networks, n_runs, path,name)
    
   
    input_folder = splitpath(path)[end]
    base_path = rsplit(path, input_folder)[1]
    
    #making save directory
    dir = "$(base_path)Results/$(name)"

    isdir(dir) || mkdir(dir)
    cd(dir)

    #copying inputs
    cp("$(path)", "Inputs",force = true)

    for i = 1:n_networks

        println("$(100*i/n_networks) %")
        inputs = ASF_Inputs.Model_Data(path); #creating parameters to run model

        rj = RegularJump(ASF_Model.density_rate,regular_c,eqs*nt)
        prob = DiscreteProblem(init_pop,inputs.Time,inputs.Parameters)
        jump_prob = JumpProblem(prob,Direct(),rj) 
        ensemble_prob_m = EnsembleProblem(jump_prob)


        outputs = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= n_runs,dt=1)
        stats = one_population_analysis(outputs) #stats for each run
        writedlm( "Results_$(i).csv",  stats, ',')
    end
        

end

sensitivity3 (generic function with 1 method)

In [69]:
function one_population_analysis(output)
    
    stats = 6
    years_till_endemic = 3
    classes, t_steps, n_ens = size(output) 
    times = output[1].t
    n_g = classes ./ 5

    Data_store = Matrix{Float32}(undef,n_ens,stats)

    for i in 1:n_ens #looping through ensemble_run

        sol = output[i]
        data = reduce(vcat,transpose.(sol.u))

        if any(x->x <0, data)
            println("Need to Reduce Timestep")
            data[data .< 0 ] .= 0
        end
        
        s_d = data[:,1:5:end]
        e_d = data[:,2:5:end]
        i_d = data[:,3:5:end]
        r_d = data[:,4:5:end]
        c_d = data[:,5:5:end]

        disease_total = e_d + i_d + c_d #classes with disease
        disease_alive = e_d + i_d

        disease_free = s_d + r_d #classes without disease
        
        disease_sum = sum(disease_total,dims=2)
        diease_alive_sum =  sum(disease_alive,dims=2)
        disease_free_sum = sum(disease_free,dims=2)
        population_sum = diease_alive_sum + disease_free_sum
        
        #we first want to see dieout time/if the diease was endemic 
        if disease_sum[end] > 0
            #Endemic/still present at end of simulation
            Data_store[i,1] = -1
        else #dies out
            dieout_index = findfirst(==(0), disease_sum)[1]
            Data_store[i,1] = times[dieout_index]
        end

        #now lets find the maximum time
        max_time_index=findfirst(==(maximum(disease_sum)), disease_sum)[1]
        Data_store[i,2] = times[max_time_index]

        #Endemic prevalence and groups alive
        if Data_store[i,1] == -1 #if endemic
            prevalence = 100*(diease_alive_sum ./ population_sum)
            e_prev = prevalence[365*years_till_endemic:end]
            Data_store[i,3] = mean(e_prev)
        else
            Data_store[i,3] = 0
        end

        #now looking at the number of groups alive at either dieout time or end if endemic
        exposed_groups = sum(disease_total, dims = 1)
        n_exposed = count(>(0), exposed_groups)
        Data_store[i,4] = 100*n_exposed/n_g
        
        population_decline = 100(1-minimum(population_sum)/population_sum[1])
        Data_store[i,5] = population_decline
        
        
        #Now want to look at minimum number of groups alive

        mimum_alive = minimum(sum((disease_alive + disease_free) .> 0, dims = 2))
        Data_store[i,6] = 100*mimum_alive/n_g
    end
    
    return Data_store

end


one_population_analysis (generic function with 1 method)

## Running Multiple Networks

In [71]:
sensitivity3(10,10,input_path,"ONE_5")

10.0 %
Watts Strogatz Small Worlds Network
Random: 1
Warning 1 disconnected feral groups!
20.0 %
Watts Strogatz Small Worlds Network
Random: 1
Warning 1 disconnected feral groups!
30.0 %
Watts Strogatz Small Worlds Network
Random: 1
Warning 1 disconnected feral groups!
40.0 %
Watts Strogatz Small Worlds Network
Random: 1
Warning 1 disconnected feral groups!
50.0 %
Watts Strogatz Small Worlds Network
Random: 1
Warning 1 disconnected feral groups!
60.0 %
Watts Strogatz Small Worlds Network
Random: 1
Warning 1 disconnected feral groups!
70.0 %
Watts Strogatz Small Worlds Network
Random: 1
Warning 1 disconnected feral groups!
80.0 %
Watts Strogatz Small Worlds Network
Random: 1
Warning 1 disconnected feral groups!
90.0 %
Watts Strogatz Small Worlds Network
Random: 1
Warning 1 disconnected feral groups!
100.0 %
Watts Strogatz Small Worlds Network
Random: 1
Warning 1 disconnected feral groups!


In [ ]:
Threads.nthreads()

In [ ]:
function sensitivity(n_networks, n_dist, n_runs, path, name)
    
    input_folder = splitpath(path)[end]
    base_path = rsplit(path, input_folder)[1]

    #making save directory
    dir = "$(base_path)Results/$(name)"

    isdir(dir) || mkdir(dir)
    cd(dir)

    #copying inputs
    cp("$(path)", "Inputs",force = true)
    
    for i = 1:n_networks
        println("$(100*i/n_networks) %")
        inputs = ASF_Inputs.Model_Data(path); #creating parameters to run model
        for j = 1:n_dist

            rj = RegularJump(ASF_Model.density_rate,regular_c,eqs*nt)
            prob = DiscreteProblem(inputs.U0,inputs.Time,inputs.Parameters)
            jump_prob = JumpProblem(prob,Direct(),rj) 
            ensemble_prob_m = EnsembleProblem(jump_prob)
            outputs = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= n_runs,EnsembleThreads(),dt=1);

            summary = ASF_Analysis.quick_analysis(outputs, inputs) #update to new function from module (ASF_Model.quick_analysis(outputs))

            writedlm( "Results_$(i)_$(j).csv",  summary, ',')
            ASF_Model.reparam!(inputs) #re-doing params with
        end
        

    end
    re
end

In [ ]:
function sensitivity2(n_networks, n_dist, n_runs, path)
    sumss = Vector{Float64}(undef, n_networks*n_dist*n_runs)
    println(n_runs)
    println(n_dist)
    println(n_networks)
    for i = 1:n_networks
        println("$(100*i/n_networks) %")
        inputs = ASF_Inputs.Model_Data(path); #creating parameters to run model

        inputs.U0[1:5:end] += (inputs.U0[2:5:end] +inputs.U0[3:5:end])
        inputs.U0[2:5:end] .= 0;
        inputs.U0[3:5:end] .= 0;

        for j = 1:n_dist

            rj = RegularJump(ASF_Model.density_rate,regular_c,eqs*nt)
            prob = DiscreteProblem(inputs.U0,inputs.Time,inputs.Parameters)
            jump_prob = JumpProblem(prob,Direct(),rj) 
            ensemble_prob_m = EnsembleProblem(jump_prob)
            outputs = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= n_runs,EnsembleThreads(),dt=1);

            #summary = ASF_Analysis.quick_analysis(outputs, inputs) #update to new function from module (ASF_Model.quick_analysis(outputs))

            for k = 1:n_runs
                final_p = sum(outputs[k].u[end])/sum(inputs.U0)
                sumss[n_networks*(i-1)+k] = final_p
            end
            ASF_Model.reparam!(inputs) #re-doing params with
        end
        

    end
    return sumss
end

In [ ]:
sensitivity(10,10,10,Linear, "ET") # running 

## Analysing Output

In [ ]:
density = ASF_Analysis.read_outputs("/home/callum/ASF/Results/Baseline//");
connect_5 = ASF_Analysis.read_outputs("/home/callum/ASF/Results/Connect_5//");


In [ ]:
density_r = ASF_Analysis.three_statistics(density,20, p_out = true);

In [ ]:
connect_5_r = ASF_Analysis.three_statistics(connect_5,20, p_out = true);

In [ ]:
using DifferentialEquations, BenchmarkTools

In [ ]:
function lorenz(u,p,t)
    dx = 10.0*(u[2]-u[1])
    dy = u[1]*(28.0-u[3]) - u[2]
    dz = u[1]*u[2] - (8/3)*u[3]
    [dx,dy,dz]
   end

In [ ]:
u0 = [1.0;0.0;0.0]
tspan = (0.0,100.0)
prob = ODEProblem(lorenz,u0,tspan)
@benchmark solve(prob,Tsit5())

In [ ]:
u0 = [1.0;0.0;0.0]
tspan = (0.0,100.0)
prob = ODEProblem(lorenz,u0,tspan)
@benchmark solve(prob,Tsit5(),save_everystep=false)

In [ ]:
function SEIRC_ODE!(du,u,p,t)
    
    ref_density = 3
    
    u[u.<0].=0 
    
    S = u[1:5:end]
    E = u[2:5:end]
    I = u[3:5:end]
    R = u[4:5:end]
    C = u[5:5:end]
  
    N = S + E + I + R + C .+ 0.0001
    Np = S + E + I + R
    
    Pops = p.Populations 
    
    tp = Pops.cum_sum[end]

    beta = copy(p.β)
    
   
    #calculating density of region
    #for i in 1:Pops.pop
    #    nf = Pops.feral[i] #number of feral groups in region
    #    ncs = Pops.cum_sum[i] #cumsum of farm and ferals over all regions
    N_feral = sum(Np) #total feral population in region i
    Density = N_feral/Pops.area[1]
    beta = beta * Density/ref_density
   #end

    column(i) = N .+ N[i]
    populations  = hcat([column(i) for i=1:tp]...)
    populations[diagind(populations)] = N;

    connected_pops = p.β_b * Np

    #procceses 
    Births = p.μ_b .* Np
    Births[(p.μ_c .== 1) .& (Np .> 0)] .= 0 #preventing boar populations growing larger than one!
    Births[(Np .== 0) .& (connected_pops .>2)] .= mean(p.μ_b)*2 #allowing migration births if neighbouring groups have pop
    

    
    du[1:5:end] = Births - ((beta.* S) ./ populations) * (I + p.ω .* C) - p.μ_d .* S + (p.μ_b-p.μ_d)./tanh(1).*S.*tanh.(Np./p.μ_c)  + p.κ .* R #S
    du[2:5:end] = ((beta.* S) ./ populations) * (I + p.ω .* C) - p.ζ .* E - p.μ_d .* E + (p.μ_b-p.μ_d)./tanh(1).*E.*tanh.(Np./p.μ_c) #E
    du[3:5:end] = p.ζ .* E - p.γ .* I -  p.μ_d .* I+ (p.μ_b-p.μ_d)./tanh(1).*I.*tanh.(Np./p.μ_c) #I
    du[4:5:end] = p.γ .* (1 .- p.ρ) .* I - p.μ_d .* R + (p.μ_b-p.μ_d)./tanh(1).*R.*tanh.(Np./p.μ_c) - p.κ .* R  #R
    du[5:5:end] = p.ρ .* p.γ .* I + p.μ_d .* I+ (p.μ_b-p.μ_d)./tanh(1).*I.*tanh.(Np./p.μ_c) - p.λ .* C#C
    nothing
end

In [ ]:
input_1 = ASF_Inputs.Model_Data(input_path); #creating parameters to run model
prob = ODEProblem(SEIRC_ODE!,input_1.U0,(0.0,10*365.0),input_1.Parameters);



In [ ]:
sol=solve(prob, AutoTsit5(Rosenbrock23()),saveat=7);

In [ ]:
@benchmark solve(prob, AutoTsit5(Rosenbrock23()),saveat=7)


In [ ]:
@benchmark solve(prob, AutoTsit5(Rosenbrock23()),saveat=1)


In [ ]:
@benchmark solve(prob, Tsit5(),saveat=1)


In [ ]:
p = [β, β_b,β_d,μ_b,μ_d,μ_c,ζ,γ,ω,ρ,λ,κ, pops]; 



In [ ]:
β = input_1.Parameters.β
β_b = input_1.Parameters.β_b
β_d= input_1.Parameters.β_d
μ_b= input_1.Parameters.μ_b
μ_d= input_1.Parameters.μ_d
μ_c= input_1.Parameters.μ_c
ζ= input_1.Parameters.ζ
γ= input_1.Parameters.γ
ω= input_1.Parameters.ω
ρ= input_1.Parameters.ρ
λ= input_1.Parameters.λ
κ= input_1.Parameters.κ
pops= input_1.Parameters.Populations


In [ ]:
input_1.Parameters.Pops